In [ ]:
import os
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim

# import data

In [ ]:
df = pd.read_pickle('../data/df_train.pkl.gzip', compression = 'gzip')

In [ ]:
df.head()

In [ ]:
df.shape

# Prepare & Check data 

In [ ]:
X = df.drop(['label','target','trace_id','E','N','Z'],axis=1)
Y = df['target']

print('X : ')
print(X.count())
print()
print('Y: ')
print(Y.count())

Y_labels = Y
Y_count_labels = len(set(Y))

print()
print('Distinct classes :', Y_count_labels)

# Train-Test Split

In [ ]:
X_train = pd.read_pickle('./data/df_train.pkl.gzip', compression='gzip')[['E','N','Z']]
X_test = pd.read_pickle('./data/df_test.pkl.gzip', compression='gzip')[['E','N','Z']]
y_train = pd.read_pickle('./data/df_train.pkl.gzip', compression='gzip')['target']
y_test = pd.read_pickle('./data/df_test.pkl.gzip', compression='gzip')['target']

In [ ]:
print('X_train : ', X_train.shape)
print('X_test: ', X_test.shape)
print('Y_train: ', y_train.shape)
print('Y_test: ', y_test.shape)

In [ ]:
y_test

# Preperation for PyTorch

In [ ]:
X_train = np.stack([np.stack([e,n,z], axis=1)
                    for e,n,z in zip(X_train['E'],X_train['N'], X_train['Z']) ], axis=0)
X_test  = np.stack([np.stack([e,n,z], axis=1)
                    for e,n,z in zip(X_test['E'],X_test['N'], X_test['Z']) ], axis=0)

In [ ]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train.to_numpy())
y_test = torch.from_numpy(y_test.to_numpy())

In [ ]:
X_train = X_train.transpose(1, 2)
X_test = X_test.transpose(1, 2)

In [ ]:
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
print('X_train : ', X_train.shape)
print('X_test: ', X_test.shape)
print('Y_train: ', y_train.shape)
print('Y_test: ', y_test.shape)

# Deep Learning Model
## Create Model

In [ ]:
c = 32 #number of channerls per conv layer
k_size = 3 #size of the convolution kernel
depth = 7

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convs = nn.ModuleList([nn.Conv1d(3, c, kernel_size=k_size, padding=1, stride=2)])
        self.convs.extend([nn.Conv1d(c, c, kernel_size=k_size, padding=1, stride=2)
                           for _ in range(10)])
        self.fc1 = nn.Linear(96, 48)
        self.fc2 = nn.Linear(48, 7)
    
    def forward(self, x):
        for i, l in enumerate(self.convs):
            x = torch.relu(self.convs[i](x))
        x = x.view(-1, 96)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

network = Net()

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv1d):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.zeros_(m.bias)

network.apply(weights_init)

In [ ]:
network(X_train[0:1])

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=1e-4)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

batch_size = 2000

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
n_epochs = 20

for epoch in range(n_epochs):
    for i, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        out = network(X)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            out_test = network(X_test)
            val_loss = loss_fn(out_test, y_test)
        # if (i+1) % 300 == 0:
        print(f'Epoch {epoch}, Step {i}, Loss {loss.item()}, Val loss {val_loss.item()}')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
with torch.no_grad():
    y_pred = network(X_test)

y_pred_cls = y_pred.max(dim=1)[1]

In [ ]:
confusion_matrix(y_test, y_pred_cls)

In [ ]:
print(classification_report(y_test, y_pred_cls))